# AWS Glue Studio Notebook
##### You are now running a AWS Glue Studio notebook; To start using your notebook you need to start an AWS Glue Interactive Session.


#### Optional: Run this cell to see available notebook commands ("magics").


In [ ]:
%help

####  Run this cell to set up and start your interactive session.


In [11]:
%idle_timeout 2880
%glue_version 4.0
%worker_type G.1X
%number_of_workers 5

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue.dynamicframe import DynamicFrame
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

You are already connected to a glueetl session 6a1837ef-0b77-4565-bd7c-2eb663fa39ea.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Current idle_timeout is 2880 minutes.
idle_timeout has been set to 2880 minutes.


You are already connected to a glueetl session 6a1837ef-0b77-4565-bd7c-2eb663fa39ea.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Setting Glue version to: 4.0


You are already connected to a glueetl session 6a1837ef-0b77-4565-bd7c-2eb663fa39ea.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Previous worker type: G.1X
Setting new worker type to: G.1X


You are already connected to a glueetl session 6a1837ef-0b77-4565-bd7c-2eb663fa39ea.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Previous number of workers: 5
Setting new number of workers to: 5



In [2]:
university_ranking_source = glueContext.create_dynamic_frame.from_catalog(
    database="univ_ranking",
    table_name="university_ranking_csv",
    transformation_ctx="UniversityRanking_Source"
)

In [3]:
university_ranking_source.show(5)

{"university": "Massachusetts Institute of Technology (MIT) ", "year": "2017", "rank_display": "1", "score": "100", "link": "https://www.topuniversities.com/universities/massachusetts-institute-technology-mit", "country": "United States", "city": "Cambridge", "region": "North America", "logo": "https://www.topuniversities.com/sites/default/files/massachusetts-institute-of-technology-mit_410_small.jpg", "type": "Private", "research_output": "Very High", "student_faculty_ratio": "4", "international_students": "3,730", "size": "M", "faculty_count": "3,065"}
{"university": "Stanford University", "year": "2017", "rank_display": "2", "score": "98.7", "link": "https://www.topuniversities.com/universities/stanford-university", "country": "United States", "city": "Stanford", "region": "North America", "logo": "https://www.topuniversities.com/sites/default/files/stanford-university_573_small.jpg", "type": "Private", "research_output": "Very High", "student_faculty_ratio": "3", "international_stu

In [4]:
# to check in the dataframe 
rankings_df = university_ranking_source.toDF()

/opt/amazon/spark/python/lib/pyspark.zip/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.


In [5]:
rankings_df.show(5)

+--------------------+----+------------+-----+--------------------+--------------+---------+-------------+--------------------+-------+---------------+---------------------+----------------------+----+-------------+
|          university|year|rank_display|score|                link|       country|     city|       region|                logo|   type|research_output|student_faculty_ratio|international_students|size|faculty_count|
+--------------------+----+------------+-----+--------------------+--------------+---------+-------------+--------------------+-------+---------------+---------------------+----------------------+----+-------------+
|Massachusetts Ins...|2017|           1|  100|https://www.topun...| United States|Cambridge|North America|https://www.topun...|Private|      Very High|                    4|                 3,730|   M|        3,065|
| Stanford University|2017|           2| 98.7|https://www.topun...| United States| Stanford|North America|https://www.topun...|Private| 

In [6]:
# Create temp view 
rankings_df.createOrReplaceTempView("university_ranking")

In [7]:
spark.sql("""SELECT university,year,rank_display,region,country FROM university_ranking""").show(5)

+--------------------+----+------------+-------------+--------------+
|          university|year|rank_display|       region|       country|
+--------------------+----+------------+-------------+--------------+
|Massachusetts Ins...|2017|           1|North America| United States|
| Stanford University|2017|           2|North America| United States|
|  Harvard University|2017|           3|North America| United States|
|University of Cam...|2017|           4|       Europe|United Kingdom|
|California Instit...|2017|           5|North America| United States|
+--------------------+----+------------+-------------+--------------+
only showing top 5 rows


In [8]:
spark.sql("""SELECT university,year,rank_display, int(rank_display) as n_rank,region,country 
FROM university_ranking""").show(5)

+--------------------+----+------------+------+-------------+--------------+
|          university|year|rank_display|n_rank|       region|       country|
+--------------------+----+------------+------+-------------+--------------+
|Massachusetts Ins...|2017|           1|     1|North America| United States|
| Stanford University|2017|           2|     2|North America| United States|
|  Harvard University|2017|           3|     3|North America| United States|
|University of Cam...|2017|           4|     4|       Europe|United Kingdom|
|California Instit...|2017|           5|     5|North America| United States|
+--------------------+----+------------+------+-------------+--------------+
only showing top 5 rows


In [9]:
# Let's Store the results data frame

clean_rankings_df = spark.sql("""SELECT 
    university,
    coalesce(int(year),9999) as year,
    rank_display,
    coalesce(int(split(rank_display,'-')[0]),9999) as n_rank,
    coalesce(float(score),-1) as score,
    country, city, region, type,
    research_output,
    coalesce(float(student_faculty_ratio),-1) as student_faculty_ratio,
    coalesce(int(regexp_replace(international_students,'[.,]','')),-1) as international_students,
    size,
    coalesce(int(regexp_replace(faculty_count,'[.,]','')),-1) as faculty_count
    FROM university_ranking""")

In [12]:
# Convert back to DynamicFrame
clean_dynamic_frame = DynamicFrame.fromDF(clean_rankings_df, glueContext, "university_ranking_clean")


## store it back to S3 

In [13]:
# Store the clean data back to S3
university_ranking_destn = glueContext.write_dynamic_frame.from_options(
    frame=clean_dynamic_frame,
    connection_type="s3",
    format="csv",
    connection_options={
        "path": "s3://aws-glue-catalog001/university_ranking/csv_clean",
        "partitionKeys": [],
    },
    transformation_ctx="university_ranking_dest",
)

In [14]:
job.commit()